## Universidad del Valle de Guatemala
### Data Science
#### Juan Marroquin 19845
#### Carlos Raxtum 19721

In [1]:
import pandas as pd

###### functions that rename and convert the data

In [2]:
# rename columns
def rename_columns(df):
    output = df.rename(
        columns = {
            'Province/State':'subregion',
            'Country/Region': 'country',
            'Lat': 'lat',
            'Long':'long',
        }
    )
    return output

In [3]:
def melt_data(df,new_name):
    output = df.melt(id_vars = ['country','subregion','lat','long'],
                     var_name = 'date_Raw',
                     value_name = new_name
    )
    return output

In [4]:
def convert_dates(df):
    output = df.assign(
        date = pd.to_datetime(df.date_Raw,format='%m/%d/%y')
    )
    output.drop(columns = ['date_Raw'],inplace = True)
    return output

In [5]:
def arrange_data(df,new_name):
    output = df.filter(['country','subregion','date','lat','long',new_name]).sort_values(['country','subregion','date','lat','long']).reset_index(drop=True)
    return output

In [6]:
def get_csv(url,value_var_name):
    df = pd.read_csv(url)
    df = rename_columns(df)
    df = melt_data(df,value_var_name)
    df = convert_dates(df)
    df = arrange_data(df,value_var_name)
    return df

##### Get data from url

In [7]:
#get case confirmed data
url ='https://data.humdata.org/hxlproxy/api/data-preview.csv?url=https%3A%2F%2Fraw.githubusercontent.com%2FCSSEGISandData%2FCOVID-19%2Fmaster%2Fcsse_covid_19_data%2Fcsse_covid_19_time_series%2Ftime_series_covid19_confirmed_global.csv&filename=time_series_covid19_confirmed_global.csv'
case_confirmed = get_csv(url,'confirmed')
# get death confirmed data
url = 'https://data.humdata.org/hxlproxy/api/data-preview.csv?url=https%3A%2F%2Fraw.githubusercontent.com%2FCSSEGISandData%2FCOVID-19%2Fmaster%2Fcsse_covid_19_data%2Fcsse_covid_19_time_series%2Ftime_series_covid19_deaths_global.csv&filename=time_series_covid19_deaths_global.csv'
dead_confirmed = get_csv(url,'death')
#get recoverd confirmed data
url = 'https://data.humdata.org/hxlproxy/api/data-preview.csv?url=https%3A%2F%2Fraw.githubusercontent.com%2FCSSEGISandData%2FCOVID-19%2Fmaster%2Fcsse_covid_19_data%2Fcsse_covid_19_time_series%2Ftime_series_covid19_recovered_global.csv&filename=time_series_covid19_recovered_global.csv'
recoverd_confirmed = get_csv(url,'recovered')

In [8]:
print('case confirmed: ',case_confirmed.shape)
print('dead confirmed: ',dead_confirmed.shape)
print('recoverd confirmed: ',recoverd_confirmed.shape)

case confirmed:  (289578, 6)
dead confirmed:  (289578, 6)
recoverd confirmed:  (274548, 6)


###### drop columns before merge

In [9]:
dead_confirmed.drop(columns=['lat','long'],inplace=True)
recoverd_confirmed.drop(columns=['lat','long'],inplace=True)

##### Merge the dfs into one

In [10]:
merge_data = case_confirmed.merge(dead_confirmed, on = ['country','subregion','date'],how = 'left').merge(recoverd_confirmed, on = ['country','subregion','date'],how = 'left')

In [11]:
merge_data.head()

,country,subregion,date,lat,long,confirmed,death,recovered
0,Afghanistan,NaN,2020-01-22,33.93911,67.709953,0,0,0.0
1,Afghanistan,NaN,2020-01-23,33.93911,67.709953,0,0,0.0
2,Afghanistan,NaN,2020-01-24,33.93911,67.709953,0,0,0.0
3,Afghanistan,NaN,2020-01-25,33.93911,67.709953,0,0,0.0
4,Afghanistan,NaN,2020-01-26,33.93911,67.709953,0,0,0.0


In [12]:
df = pd.read_csv('2022_population.csv',encoding='utf-8')

In [13]:
# change name in for the US
df.loc[df['country'] == 'United States', 'country'] = 'US'
merge_data.loc[merge_data['country'] == 'Korea, North', 'country'] = 'North Korea'
merge_data.loc[merge_data['country'] == 'Korea, South', 'country'] = 'South Korea'
merge_data.loc[merge_data['country'] == "Cote d'Ivoire", 'country'] = 'Ivory Coast'
merge_data.loc[merge_data['country'] == "Czechia", 'country'] = 'Czech Republic'
merge_data.loc[merge_data['country'] == "Cabo Verde", 'country'] = 'Cape Verde'
merge_data.loc[merge_data['country'] == "Taiwan*", 'country'] = 'Taiwan'
merge_data.loc[merge_data['country'] == "Holy See", 'country'] = 'Vatican City'
#merge_data.loc[merge_data['country'] == "Samoa", 'country'] = 'American Samoa'
#merge_data.loc[merge_data['country'] == "Martinica", 'country'] = 'Martinique'

In [14]:
#merge_data.to_csv('covid19.csv',index = False, encoding='utf-8')

In [15]:
df.drop(['iso_code', 'rank','world_%','growth_rate','2020_population'], axis = 1, inplace = True) 

In [16]:
df = df.rename(columns={'2022_last_updated': 'population'})

In [17]:
df["population"] = df["population"].map(lambda x: x.replace(',', ''))
df["area_sq_km"] = df["area_sq_km"].map(lambda x: x.replace(',', ''))
df["density_sq_km"] = df["density_sq_km"].map(lambda x: x.replace('/sq_km', '').replace(',', ''))

In [18]:
df.head()

,country,population,area_sq_km,density_sq_km
0,China,1425865702,9706961,147
1,India,1419924173,3287590,431
2,US,338737868,9372610,36
3,Indonesia,276002970,1904569,145
4,Pakistan,237129351,881912,267


In [19]:
df["area_sq_km"] = df["area_sq_km"].astype(float)
df["population"] = df["population"].astype(float)
df["density_sq_km"] = df["density_sq_km"].astype(float)

In [20]:
df.head()

,country,population,area_sq_km,density_sq_km
0,China,1.425866e+09,9706961.0,147.0
1,India,1.419924e+09,3287590.0,431.0
2,US,3.387379e+08,9372610.0,36.0
3,Indonesia,2.760030e+08,1904569.0,145.0
4,Pakistan,2.371294e+08,881912.0,267.0


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216 entries, 0 to 215
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   country        216 non-null    object 
 1   population     216 non-null    float64
 2   area_sq_km     216 non-null    float64
 3   density_sq_km  216 non-null    float64
dtypes: float64(3), object(1)
memory usage: 6.9+ KB


In [26]:
df_list = df.country.unique().tolist()
merge_list = merge_data.country.unique().tolist()

In [28]:
drop_countries = []
for name in df_list:
    if name not in merge_list:
        drop_countries.append(name)

In [29]:
for name in drop_countries:
    df = df.drop(df[df.country == name ].index)
    df.reset_index(drop=True)

In [30]:
df2 = pd.merge(merge_data, df, on=['country'])

In [31]:
df2.head()

,country,subregion,date,lat,long,confirmed,death,recovered,population,area_sq_km,density_sq_km
0,Afghanistan,NaN,2020-01-22,33.93911,67.709953,0,0,0.0,41439625.0,652230.0,63.0
1,Afghanistan,NaN,2020-01-23,33.93911,67.709953,0,0,0.0,41439625.0,652230.0,63.0
2,Afghanistan,NaN,2020-01-24,33.93911,67.709953,0,0,0.0,41439625.0,652230.0,63.0
3,Afghanistan,NaN,2020-01-25,33.93911,67.709953,0,0,0.0,41439625.0,652230.0,63.0
4,Afghanistan,NaN,2020-01-26,33.93911,67.709953,0,0,0.0,41439625.0,652230.0,63.0


In [32]:
df2.to_csv('covid19.csv',index = False, encoding='utf-8')